In [8]:
# 네이버 후기 크롤링

import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import os
import subprocess
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, TimeoutException

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_user_input():
    url = input("상품 페이지 URL을 입력하세요: ")
    file_name = input("저장할 CSV 파일 이름을 입력하세요: ")
    
    while True:
        try:
            max_reviews = int(input("크롤링할 최대 리뷰 수를 입력하세요: "))
            if max_reviews > 0:
                break
            else:
                logging.warning("0보다 큰 정수를 입력해주세요.")
        except ValueError:
            logging.warning("올바른 정수를 입력해주세요.")
    
    return url, file_name, max_reviews

def initialize_webdriver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def random_wait(min_time=2, max_time=4):
    time.sleep(random.uniform(min_time, max_time))

def extract_review_info(driver, reviews_data, max_reviews, file_name):
    page_num = 1
    while len(reviews_data) < max_reviews:
        try:
            logging.info(f"현재 페이지: {page_num}")
            
            # 리뷰 목록이 로드될 때까지 기다림
            reviews = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.BnwL_cs1av'))
            )
            
            logging.info(f"현재 페이지에서 발견된 리뷰 수: {len(reviews)}")
            
            # 현재 페이지에서 리뷰 추출
            for review in reviews:
                if len(reviews_data) >= max_reviews:
                    break
                try:
                    rating = review.find_element(By.CSS_SELECTOR, 'em._15NU42F3kT').text
                    user_id = review.find_element(By.CSS_SELECTOR, 'strong._2L3vDiadT9').text
                    date = review.find_element(By.CSS_SELECTOR, 'div.iWGqB6S4Lq span._2L3vDiadT9').text
                    option = review.find_element(By.CSS_SELECTOR, 'div._2FXNMst_ak').text
                    review_text = review.find_element(By.CSS_SELECTOR, 'div._1kMfD5ErZ6 span._2L3vDiadT9').text
                    
                    reviews_data.append({
                        "평점": rating,
                        "구매자아이디": user_id,
                        "작성일": date,
                        "선택옵션명": option,
                        "리뷰내용": review_text
                    })
                    logging.info(f"리뷰 {len(reviews_data)}: {user_id} - {rating}")
                except NoSuchElementException as e:
                    logging.warning(f"리뷰 요소를 찾을 수 없습니다. 오류: {e}")
                    continue

            # 중간 저장
            if len(reviews_data) % 100 == 0:
                save_to_csv(reviews_data, f"{file_name}_interim")
                logging.info(f"중간 저장 완료: {len(reviews_data)}개의 리뷰")

            # 다음 페이지로 이동
            try:
                next_page_buttons = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.UWN4IvaQza._nlog_click[aria-current="false"]'))
                )
                
                next_page_button = None
                for button in next_page_buttons:
                    button_number = int(button.text)
                    if button_number > page_num:
                        next_page_button = button
                        break
                
                if next_page_button:
                    driver.execute_script("arguments[0].click();", next_page_button)
                    page_num = button_number
                    random_wait()
                else:
                    logging.info("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지일 수 있습니다.")
                    break
            except (TimeoutException, NoSuchElementException) as e:
                logging.error(f"다음 페이지 버튼을 찾을 수 없습니다. 오류: {e}")
                break

        except TimeoutException:
            logging.error("페이지 로드 시간이 초과되었습니다. 다음 페이지로 넘어갑니다.")
            page_num += 1
            continue
        except NoSuchWindowException:
            logging.error("브라우저 창이 닫혔습니다. 프로그램을 종료합니다.")
            break
        except Exception as e:
            logging.error(f"예상치 못한 오류가 발생했습니다: {e}")
            break
    
    logging.info(f"총 {len(reviews_data)}개의 리뷰를 수집했습니다.")

def save_to_csv(reviews_data, file_name):
    df = pd.DataFrame(reviews_data)
    df.index += 1
    full_path = os.path.abspath(f"{file_name}.csv")
    df.to_csv(full_path, index_label="순번", encoding="utf-8-sig")
    logging.info(f"{full_path} 파일에 저장되었습니다.")
    return full_path

def open_file(file_path):
    if os.name == 'nt':  # Windows
        os.startfile(file_path)
    elif os.name == 'posix':  # macOS and Linux
        if sys.platform == 'darwin':  # macOS
            subprocess.call(('open', file_path))
        else:  # Linux
            subprocess.call(('xdg-open', file_path))

def main():
    url, file_name, max_reviews = get_user_input()
    driver = initialize_webdriver()
    driver.get(url)
    
    try:
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-name="REVIEW"]'))
        )
        review_tab.click()
        random_wait()
    except (NoSuchElementException, TimeoutException):
        logging.error("리뷰 탭을 찾을 수 없습니다.")
        driver.quit()
        return

    reviews_data = []
    try:
        extract_review_info(driver, reviews_data, max_reviews, file_name)
    finally:
        # 프로그램이 중단되더라도 현재까지의 데이터 저장
        if reviews_data:
            file_path = save_to_csv(reviews_data, file_name)
            logging.info(f"프로그램 종료. 수집된 {len(reviews_data)}개의 리뷰를 저장했습니다.")
        driver.quit()

    # 파일 자동으로 열기
    try:
        open_file(file_path)
        logging.info(f"{file_path} 파일이 자동으로 열렸습니다.")
    except Exception as e:
        logging.error(f"파일을 자동으로 열 수 없습니다. 오류: {e}")
        logging.info(f"파일 경로: {file_path}")

if __name__ == "__main__":
    main()

상품 페이지 URL을 입력하세요: https://smartstore.naver.com/tgesil/products/9925194725
저장할 CSV 파일 이름을 입력하세요: 청송_패브릭라벨모음
크롤링할 최대 리뷰 수를 입력하세요: 328


2024-09-13 14:41:20,607 - INFO - ====== WebDriver manager ======
2024-09-13 14:41:21,846 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-13 14:41:22,288 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-13 14:41:22,774 - INFO - Driver [C:\Users\owner\.wdm\drivers\chromedriver\win64\128.0.6613.137\chromedriver-win32/chromedriver.exe] found in cache
2024-09-13 14:41:28,119 - INFO - 현재 페이지: 1
2024-09-13 14:41:28,130 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-13 14:41:28,190 - INFO - 리뷰 1: tj_w******** - 5
2024-09-13 14:41:28,233 - INFO - 리뷰 2: imp7*** - 5
2024-09-13 14:41:28,283 - INFO - 리뷰 3: rlas******** - 5
2024-09-13 14:41:28,325 - INFO - 리뷰 4: kim_******** - 5
2024-09-13 14:41:28,369 - INFO - 리뷰 5: nsu0**** - 5
2024-09-13 14:41:28,412 - INFO - 리뷰 6: mode***** - 5
2024-09-13 14:41:28,458 - INFO - 리뷰 7: thdt****** - 5
2024-09-13 14:41:28,500 - INFO - 리뷰 8: thdt****** - 5
2024-09-13 14:41:28,544 - INFO - 리뷰 9: thdt****** - 5
2024-09-13 14:41:28,587 - INFO

2024-09-13 14:41:49,021 - INFO - 리뷰 131: kimm***** - 5
2024-09-13 14:41:49,063 - INFO - 리뷰 132: kimm***** - 5
2024-09-13 14:41:49,104 - INFO - 리뷰 133: kimm***** - 5
2024-09-13 14:41:49,145 - INFO - 리뷰 134: tkdb***** - 5
2024-09-13 14:41:49,187 - INFO - 리뷰 135: khk8*** - 5
2024-09-13 14:41:49,228 - INFO - 리뷰 136: shfk***** - 5
2024-09-13 14:41:49,271 - INFO - 리뷰 137: supr*** - 4
2024-09-13 14:41:49,309 - INFO - 리뷰 138: joki**** - 5
2024-09-13 14:41:49,351 - INFO - 리뷰 139: 3sta***** - 5
2024-09-13 14:41:49,394 - INFO - 리뷰 140: 3sta***** - 5
2024-09-13 14:41:51,529 - INFO - 현재 페이지: 8
2024-09-13 14:41:51,536 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-13 14:41:51,578 - INFO - 리뷰 141: 3sta***** - 5
2024-09-13 14:41:51,621 - INFO - 리뷰 142: kjs9*** - 4
2024-09-13 14:41:51,661 - INFO - 리뷰 143: 3sta***** - 5
2024-09-13 14:41:51,704 - INFO - 리뷰 144: dusw***** - 5
2024-09-13 14:41:51,744 - INFO - 리뷰 145: dusw***** - 5
2024-09-13 14:41:51,785 - INFO - 리뷰 146: dusw***** - 5
2024-09-13 14:41:51,827 - INF

2024-09-13 14:42:14,076 - INFO - 리뷰 266: km**** - 5
2024-09-13 14:42:14,120 - INFO - 리뷰 267: km**** - 5
2024-09-13 14:42:14,163 - INFO - 리뷰 268: km**** - 5
2024-09-13 14:42:14,205 - INFO - 리뷰 269: hee5*** - 5
2024-09-13 14:42:14,248 - INFO - 리뷰 270: to**** - 5
2024-09-13 14:42:14,289 - INFO - 리뷰 271: bnh_**** - 5
2024-09-13 14:42:14,333 - INFO - 리뷰 272: fu**** - 5
2024-09-13 14:42:14,375 - INFO - 리뷰 273: fu**** - 5
2024-09-13 14:42:14,425 - INFO - 리뷰 274: ultr***** - 5
2024-09-13 14:42:14,468 - INFO - 리뷰 275: ultr***** - 5
2024-09-13 14:42:14,513 - INFO - 리뷰 276: ultr***** - 5
2024-09-13 14:42:14,557 - INFO - 리뷰 277: km**** - 5
2024-09-13 14:42:14,600 - INFO - 리뷰 278: nabe**** - 5
2024-09-13 14:42:14,647 - INFO - 리뷰 279: nabe**** - 5
2024-09-13 14:42:14,691 - INFO - 리뷰 280: nabe**** - 5
2024-09-13 14:42:16,817 - INFO - 현재 페이지: 15
2024-09-13 14:42:16,825 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-13 14:42:16,870 - INFO - 리뷰 281: nabe**** - 5
2024-09-13 14:42:16,912 - INFO - 리뷰 282: nabe****

상품 페이지 URL을 입력하세요: https://brand.naver.com/airmade/products/9958621360
저장할 CSV 파일 이름을 입력하세요: 에어메이드 자동 먼지비움 진공 무선청소기
크롤링할 최대 리뷰 수를 입력하세요: 1862